In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pf
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
SIMDATAPATH = os.path.join(PATH, 'sim_data')

def TemplateFrequenciesFromSpectrum(f_start, fspace, path2tritium, hz_per_ev=49.1e3, bw_ev=100, bins_tritium=201, power=0.7):
    
    with open(path2tritium, 'rb') as infile:
        tritium_data = pkl.load(infile)
        
    vals, bins = np.histogram(tritium_data, bins_tritium)
    
    bin_prob = vals / vals.sum()
    rel_bin_prob = bin_prob / bin_prob.max()
    
    bw = abs(hz_per_ev) * bw_ev
    f_max = f_start + bw
    interpolation_frequencies = np.linspace(f_start, f_max, rel_bin_prob.size)
    rel_bin_prob_interp = scipy.interpolate.interp1d(interpolation_frequencies, rel_bin_prob, fill_value='extrapolate')
    
    f_last = f_start
    frequency_list = []

    while f_last < f_max:
        frequency_list.append(f_last)
        delta_f =  fspace / rel_bin_prob_interp(f_last) ** power
        f_last += delta_f
        
    frequency_list = np.array(frequency_list)
    
    return frequency_list

def RNGFrequenciesFromSpectrum(f_start, f_end, N, path2tritium):
    with open(path2tritium, 'rb') as infile:
        tritium_data = pkl.load(infile)
        
    #vals, bins = np.histogram(tritium_data, bins_tritium)
    
    norm_tritium_data = (tritium_data - np.min(tritium_data)) / (np.max(tritium_data) - np.min(tritium_data))
    
    rng = np.random.default_rng()
    
    rng_frequencies = f_start + (f_end - f_start) * norm_tritium_data[rng.integers(0, norm_tritium_data.size, N)]
    
    return rng_frequencies

def CalcMatch(template_f, rng_f, t):
    
    templates = np.zeros((template_f.size, t.size), dtype=np.complex64)
    
    for n in range(template_f.size):
        templates[n, :] = np.exp(1j * 2 * np.pi * template_f[n] * t)
        
    signals = np.zeros((rng_f.size, t.size), dtype=np.complex64)
    
    for n in range(rng_f.size):
        
        signals[n, :] = np.exp(1j * 2 * np.pi * rng_f[n] * t)
        
    match = abs(np.matmul(templates, signals.conjugate().T))
    
    return match
    

In [ ]:
tritium_samples = os.path.join(SIMDATAPATH, '210615_tritium_energy_spectrum.pkl')

N = 8192
fs = 200e6
freq_spacing_0 = 1 * fs / N
f_start = 10e6
n_rng = 2000
t = np.arange(0, N, 1) * 1 / fs

template_f = TemplateFrequenciesFromSpectrum(f_start, freq_spacing_0, tritium_samples, power=0.5)
print(template_f.shape)
rng_f = RNGFrequenciesFromSpectrum(f_start, template_f[-1], n_rng, tritium_samples)
#print(rng_f[0:10])

In [ ]:
matches = CalcMatch(template_f, rng_f, t)
ideal_matches = np.diag(CalcMatch(rng_f, rng_f, t))

In [ ]:
with open(tritium_samples, 'rb') as infile:
    tritium_data = pkl.load(infile)
    
tritium_data = np.array(tritium_data)
rng = np.random.default_rng()
nsample = 2000
rng_tritium_energy = tritium_data[rng.integers(0, tritium_data.size, nsample)]

sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)


hist = ax.hist(rng_tritium_energy, 64)
#hist = ax.hist(18500 + rng_f / (49.1e3 * (rng_f.max() - rng_f.min())), 64)
ax.set_xlabel('Energy (eV)')
ax.set_ylabel('N')
ax.set_title('Tritium Energy Spectrum Samples')
plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210810_dummy_energy_tritium_spectrum'))

In [ ]:
plt.plot(template_f, template_f, '.')
#plt.yscale('log')

In [ ]:
hist = plt.hist(matches.max(axis=0) / ideal_matches, 64)

In [ ]:
match_ratio = matches.max(axis=0) / ideal_matches

print(np.mean(match_ratio))

# use tritium spectrum shape to pick templates and signals

In [ ]:
freq_spacing_ratios = np.linspace(0.1, 1.5, 21)
powers = np.arange(2, 11, 1) / 10


match_ratio_matrix = np.zeros((powers.size, freq_spacing_ratios.size))

for i, power in enumerate(powers):
    for j, ratio in enumerate(freq_spacing_ratios):
        print(i, j)
        N = 8192
        fs = 200e6
        freq_spacing_0 = ratio * fs / N
        f_start = 10e6
        n_rng = 2000
        t = np.arange(0, N, 1) * 1 / fs
        
        template_f = TemplateFrequenciesFromSpectrum(f_start, freq_spacing_0, tritium_samples, power=power)
        #print(template_f.shape)
        rng_f = RNGFrequenciesFromSpectrum(f_start, template_f[-1], n_rng, tritium_samples)
        matches = CalcMatch(template_f, rng_f, t)
        ideal_matches = np.diag(CalcMatch(rng_f, rng_f, t))
        
        
        match_ratio_matrix[i, j] = np.mean(matches.max(axis=0) / ideal_matches)

In [ ]:
freq_spacing_ratios = np.linspace(0.1, 1.5, 21)
powers = np.arange(2, 11, 1) / 10


template_number = np.zeros((powers.size, freq_spacing_ratios.size))

for i, power in enumerate(powers):
    for j, ratio in enumerate(freq_spacing_ratios):
        print(i, j)
        N = 8192
        fs = 200e6
        freq_spacing_0 = ratio * fs / N
        f_start = 10e6
        n_rng = 2000
        t = np.arange(0, N, 1) * 1 / fs
        
        template_f = TemplateFrequenciesFromSpectrum(f_start, freq_spacing_0, tritium_samples, power=power)
        #print(template_f.shape)
        template_number[i, j] = template_f.size

In [ ]:
for i in range(match_ratio_matrix.shape[0]):
    
    plt.plot(freq_spacing_ratios, match_ratio_matrix[i, :],  '.', label = powers[i],)
plt.legend(loc=3)

In [ ]:
for i in range(match_ratio_matrix.shape[0]):
    
    plt.plot(freq_spacing_ratios, match_ratio_matrix[i, :], '.', label = powers[i],)
plt.legend(loc=3)
plt.xlim(0.1, 0.5)
plt.ylim(0.8, 1.0)

In [ ]:
for i in range(template_number.shape[0]):
    plt.plot(freq_spacing_ratios, template_number[i, :], label=powers[i])
plt.legend(loc=1)

# uniform templates and trtitium spectrum frequencies

In [ ]:
freq_spacing_ratios = np.linspace(0.1, 1.5, 21)
powers = np.arange(2, 11, 1) / 10


match_ratio_matrix_uniform = np.zeros((freq_spacing_ratios.size))

#for i, power in enumerate(powers):
for j, ratio in enumerate(freq_spacing_ratios):
    print(i, j)
    N = 8192
    fs = 200e6
    hz_per_ev=49.1e3
    bw_ev=100
    freq_spacing_0 = ratio * fs / N
    f_start = 10e6
    f_max = f_start + hz_per_ev * bw_ev
    n_rng = 2000
    t = np.arange(0, N, 1) * 1 / fs

    #template_f = TemplateFrequenciesFromSpectrum(f_start, freq_spacing_0, tritium_samples, power=power)
    template_f = np.arange(f_start, f_max + freq_spacing_0, freq_spacing_0)
    #print(template_f.shape)
    rng_f = RNGFrequenciesFromSpectrum(f_start, template_f[-1], n_rng, tritium_samples)
    matches = CalcMatch(template_f, rng_f, t)
    ideal_matches = np.diag(CalcMatch(rng_f, rng_f, t))


    match_ratio_matrix_uniform[j] = np.mean(matches.max(axis=0) / ideal_matches)

In [ ]:

plt.plot(freq_spacing_ratios, match_ratio_matrix_uniform,  '.', label = powers[i],)
plt.legend(loc=3)

In [ ]:
template_number_uniform = np.zeros((freq_spacing_ratios.size))


for j, ratio in enumerate(freq_spacing_ratios):
    print(i, j)
    N = 8192
    fs = 200e6
    freq_spacing_0 = ratio * fs / N
    f_start = 10e6
    f_max = f_start + hz_per_ev * bw_ev
    n_rng = 2000
    t = np.arange(0, N, 1) * 1 / fs

    #template_f = TemplateFrequenciesFromSpectrum(f_start, freq_spacing_0, tritium_samples, power=power)
    template_f = np.arange(f_start, f_max + freq_spacing_0, freq_spacing_0)
    #print(template_f.shape)
    template_number_uniform[j] = template_f.size

In [ ]:

plt.plot(freq_spacing_ratios, template_number_uniform,  '.', label = powers[i],)
plt.legend(loc=3)

# compare uniform spacing and variable spacing curves

In [ ]:
plt.figure()
for i in range(match_ratio_matrix.shape[0]):
    
    plt.plot(freq_spacing_ratios, match_ratio_matrix[i, :],  '.', label = powers[i],)


plt.plot(freq_spacing_ratios, match_ratio_matrix_uniform,  '.', label = 'uniform',)
plt.legend(loc=3)
plt.xlim(0.1, 0.4)
plt.ylim(0.8, 1.1)
plt.figure()
for i in range(template_number.shape[0]):
    plt.plot(freq_spacing_ratios, template_number[i, :], label=powers[i])


plt.plot(freq_spacing_ratios, template_number_uniform, label = 'uniform',)
plt.legend(loc=1)

plt.xlim(0.1, 0.4)

# find optimum

In [ ]:
combined_match_matrix = np.zeros((10, 21))
combined_match_matrix[0, :] = match_ratio_matrix_uniform
combined_match_matrix[1:, :] = match_ratio_matrix

combined_template_number = np.zeros((10, 21))
combined_template_number[0, :] = template_number_uniform
combined_template_number[1:, :] = template_number

In [ ]:
plt.imshow(combined_match_matrix, interpolation='none', aspect='auto')
plt.colorbar()

In [ ]:
gamma = 0.90
x_pts = np.argwhere(combined_match_matrix >= gamma)[:, 0]
y_pts = np.argwhere(combined_match_matrix >= gamma)[:, 1]

print(np.argmin(combined_template_number[x_pts, y_pts]))
print(x_pts[np.argmin(combined_template_number[x_pts, y_pts])], y_pts[np.argmin(combined_template_number[x_pts, y_pts])])
print(np.min(combined_template_number[x_pts, y_pts]))

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(combined_match_matrix, interpolation='none', aspect='auto', cmap=cmap, extent = (0.1, 1.5, 1.1, 0.1))
ax.plot(freq_spacing_ratios[x_pts[np.argmin(combined_template_number[x_pts, y_pts])]-1], powers[y_pts[np.argmin(combined_template_number[x_pts, y_pts])]-1], 'r*', markersize=16)
plt.colorbar(img,label=r'Match Ratio ($\Gamma$)')

ax.set_xlabel('R')
ax.set_ylabel(r'$\alpha$')
ax.set_title('Match Ratio for Template Positioning Functions')
ax.text(0.4, 0.8, rf'Best N for $\Gamma \geq {gamma}$ = {int(np.min(combined_template_number[x_pts, y_pts]))}', color='w')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, f'210809_match_ratio_gamma{gamma}.png'))

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)


img = ax.imshow(np.log10(combined_template_number), interpolation='none', aspect='auto', cmap=cmap, extent = (0.1, 1.5, 1.1, 0.1))
ax.plot(freq_spacing_ratios[x_pts[np.argmin(combined_template_number[x_pts, y_pts])]-1], powers[y_pts[np.argmin(combined_template_number[x_pts, y_pts])]-1], 'r*')
plt.colorbar(img, label=r'Log$_{10}(N)$')

ax.set_xlabel('R')
ax.set_ylabel(r'$\alpha$')
ax.set_title('Template Number for Positioning Functions')

# plot best N as we decrease match ratio

In [ ]:
gamma_array = np.linspace(0.99, 0.7, 25)
opt_N = np.zeros(25)
for i, gamma in enumerate(gamma_array):
    x_pts = np.argwhere(combined_match_matrix >= gamma)[:, 0]
    y_pts = np.argwhere(combined_match_matrix >= gamma)[:, 1]
    opt_N[i] = np.min(combined_template_number[x_pts, y_pts])
    
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(gamma_array, opt_N,)
#ax.set_yscale('log')
ax.set_title('Minimum Template Number to Achieve Mean Match Ratio')
ax.set_xlabel(r'Match Ratio ($\Gamma$)')
ax.set_ylabel('Template Number')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210809_min_templates_vs_mean_match.png'))

# uniform templates and uniform signals

In [ ]:
fs = 200e6
N = 8192
freq_spacing = fs/N
f1 = 10e6
f2 = 10e6 + freq_spacing * 1

prod1 = np.zeros(N)
for nsamp in np.arange(0, N, 1):
    
    t = np.arange(0, nsamp, 1) / fs

    x1_real = np.cos(2*np.pi*f1*t)
    x2_real = np.cos(2*np.pi*f2*t)

    prod1[nsamp] = abs(np.dot(x1_real, x2_real))
    
prod2 = np.zeros(N)
for nsamp in np.arange(0, N, 1):
    
    t = np.arange(0, nsamp, 1) / fs

    x1_real = np.cos(2*np.pi*f2*t)
    x2_real = np.cos(2*np.pi*f2*t)

    prod2[nsamp] = abs(np.dot(x1_real, x2_real))

In [ ]:
plt.plot(prod2)
plt.plot(prod1)

In [ ]:
plt.plot(prod1 / prod2)

In [ ]:
N = 8192
fs = 200e6
N_freq = 101
f1 = 10e6 + np.linspace(0, 0.5, N_freq) * fs/N
f2 = 10e6
t = np.arange(0, N, 1) / fs


prod3 = np.zeros(N_freq)

for i, frequency in enumerate(f1):
    
    x1_real = np.cos(2*np.pi*frequency*t)
    x2_real = np.cos(2*np.pi*f2*t)

    prod3[i] = abs(np.dot(x1_real, x2_real)) / abs(np.dot(x2_real, x2_real))
    
    
    

In [ ]:
plt.plot(np.linspace(0, 0.5, N_freq), prod3, '.')
#plt.xlim(0, 0.25)
#plt.ylim(0.6, 1.0)

In [ ]:
print(prod3[60], np.linspace(0, 0.5, N_freq)[60])

In [ ]:
# FWHM of frequency peak is 0.6 * fs /N

In [ ]:
N = 8192
fs = 200e6
print(f'FWHM = {0.6 * fs / N}')
FWHM = .2 * fs / N

In [ ]:
hz_per_ev = -49.1e3
bw_100ev = abs(hz_per_ev) * 100
print(f'100 eV BW = {bw_100ev * 1e-6} MHz')
print(f'100 eV BW = {np.int32(bw_100ev / FWHM)} FWHM')

In [ ]:
n_fwhm = np.int32(bw_100ev / FWHM)
template_frequencies = np.arange(0, n_fwhm, 1) * FWHM + 10e6

templates = np.zeros((n_fwhm, t.size), dtype=np.complex64)

for n in range(n_fwhm):
    templates[n, :] = np.exp(1j*2*np.pi*template_frequencies[n]*t)



In [ ]:
rng=np.random.default_rng()

In [ ]:
rng_frequencies = rng.uniform(template_frequencies[0], template_frequencies[-1], 2000)

In [ ]:
signals = np.zeros((rng_frequencies.size, t.size), dtype=np.complex64)

for n, f in enumerate(rng_frequencies):
    signals[n, :] = np.exp(1j*2*np.pi*rng_frequencies[n]*t)

In [ ]:
scores = abs(np.matmul(templates, signals.conjugate().T))
ideal_scores = np.diag(abs(np.matmul(signals, signals.conjugate().T)))

In [ ]:
best_template_scores = scores.max(axis=0)

In [ ]:
hist = plt.hist(best_template_scores/ideal_scores, 64)
plt.vlines(np.mean(best_template_scores/ideal_scores), 0, 50, 'r')
print(np.mean(best_template_scores/ideal_scores))
#plt.xscale('log')

In [ ]:
# calculate mean mismatch as a function of the spacing of templates in frequency using units of fs / N

In [ ]:
n_spacings = 31

frequency_spacing = np.linspace(0.2, 3, n_spacings) * fs / N
mean_match = np.zeros(n_spacings)
for i, spacing in enumerate(frequency_spacing):
    if i % 5 == 4:
        print(i + 1)
    n_spacing = np.int32(bw_100ev / spacing)
    template_frequencies = np.arange(0, n_spacing, 1) * spacing + 10e6

    templates = np.zeros((n_spacing, t.size), dtype=np.complex64)

    for n in range(n_spacing):
        templates[n, :] = np.exp(1j*2*np.pi*template_frequencies[n]*t)
        
    rng_frequencies = rng.uniform(template_frequencies[0], template_frequencies[-1], 1000)
    
    signals = np.zeros((rng_frequencies.size, t.size), dtype=np.complex64)

    for n, f in enumerate(rng_frequencies):
        signals[n, :] = np.exp(1j*2*np.pi*rng_frequencies[n]*t)
        
    scores = abs(np.matmul(templates, signals.conjugate().T))
    ideal_scores = np.diag(abs(np.matmul(signals, signals.conjugate().T)))
    best_template_scores = scores.max(axis=0)
    
    mean_match[i] = np.mean(best_template_scores/ideal_scores)

In [ ]:
sns.set_theme(context='talk', style='ticks')
clist = sns.color_palette('deep')

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)



line1 = ax.plot(frequency_spacing * N / fs, mean_match, color=clist[0], label='Match Ratio')
ax.set_xlabel(r'Frequency Spacing $(\frac{f_s}{N_{sample}})$')
ax.set_ylabel('Match Ratio')

ax2 = ax.twinx()

line2 = ax2.plot(select_freq_spacing, N_templates, color=clist[1], label=r'$N_{template}$')
ax2.grid()
ax2.set_ylabel(r'$N_{template}$')

import matplotlib

patch1 = matplotlib.patches.Patch(color=clist[0])
patch2 = matplotlib.patches.Patch(color=clist[1])

ax2.legend([patch1, patch2], ['Match Ratio', r'$N_{template}$'])

ax2.set_title(r'Mean Match Ratio and $N_{template}$ vs Freq. Spacing')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210810_match_ratio_and_Ntemplate_vs_freq_spacing_uniform_dist.png'))

In [ ]:

N_templates = []
select_freq_spacing = []
for i, match_factor in enumerate(np.concatenate(([0.99], 0.95 + np.arange(0, -55, -5)/100))):

    ind = np.argmin(abs(mean_match - match_factor))
    select_freq_spacing.append((frequency_spacing * N / fs)[ind])
    N_templates.append(np.int32(bw_100ev / (frequency_spacing)[ind]))

In [ ]:
# mean match = 0.99 -> frequency spacing = 0.29 fs / N
# mean match = 0.95 -> frequency spacing = 0.57 fs / N
# mean match = 0.90 -> frequency spacing = 0.85 fs / N
# mean match = 0.85 -> frequency spacing = 1.13 fs / N
# mean match = 0.75 -> frequency spacing = 1.51 fs / N

In [ ]:
N = 8192
fs = 200e6

freq_spacing = 1.51 * fs / N
bw_100ev = abs(hz_per_ev) * 100
print(f'100 eV BW = {bw_100ev * 1e-6} MHz')
print(f'100 eV BW requires {np.int32(bw_100ev / freq_spacing)} templates')

In [ ]:
print(-1 * freq_spacing / hz_per_ev)